In [1]:
import os
import json
import logging
import traceback
import requests
from datetime import datetime, timedelta
from pprint import pformat
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium_recaptcha_solver import RecaptchaSolver
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.chat_models import ChatOpenAI

from prompts.scheduling_prompts import scheduling_prompt
from utils.calendar_utils import (
    generate_mock_calendar,
    convert_to_unified_format,
    find_overlapping_times
)
from scrape import book_calendly_appointment
from book2 import setup_calendly_api, get_calendly_availability, create_booking_url, get_suggested_time, setup_selenium_with_recaptcha_solver


/home/zain/projects/xsap/calendly/.venv/lib/python3.12/site-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/home/zain/projects/xsap/calendly/.venv/lib/python3.12/site-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/home/zain/projects/xsap/calendly/.venv/lib/python3.12/site-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/home/zain/projects/xsap/calendly/.venv/lib/python3.12/site-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
/home/zain/projects/xsap/calendly/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffm

In [2]:
mock_calendar = generate_mock_calendar()
print(json.dumps(mock_calendar, indent=2))

{
  "invitee_publisher_error": false,
  "today": "2025-03-15",
  "availability_timezone": "America/Los_Angeles",
  "days": [
    {
      "date": "2025-03-15",
      "status": "unavailable",
      "spots": [],
      "enabled": false
    },
    {
      "date": "2025-03-16",
      "status": "unavailable",
      "spots": [],
      "enabled": false
    },
    {
      "date": "2025-03-17",
      "status": "available",
      "spots": [
        {
          "status": "available",
          "start_time": "2025-03-17T09:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T09:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T10:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T10:30:00-07:00",
          "invitees_remaining": 1
        },
   

In [3]:
uuid, profile_slug, event_type_slug = setup_calendly_api('https://calendly.com/robertjandali/30min')
range_url = f"https://calendly.com/api/booking/event_types/{uuid}/calendar/range"
start_date = datetime.now()
end_date = start_date + timedelta(days=7)
        
params = {
            "timezone": "America/Los_Angeles",
            "diagnostics": "false",
            "range_start": start_date.strftime("%Y-%m-%d"),
            "range_end": end_date.strftime("%Y-%m-%d")
        }
        
calendar_response = requests.get(range_url, params=params)
calendar_response.raise_for_status()
        
calendly_data = calendar_response.json()
print(json.dumps(calendly_data, indent=2))

{
  "invitee_publisher_error": false,
  "today": "2025-03-15",
  "availability_timezone": "America/Los_Angeles",
  "days": [
    {
      "date": "2025-03-17",
      "status": "available",
      "spots": [
        {
          "status": "available",
          "start_time": "2025-03-17T00:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T00:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T01:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T02:00:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T02:30:00-07:00",
          "invitees_remaining": 1
        },
        {
          "status": "available",
          "start_time": "2025-03-17T03:00:00-07

In [4]:
from datetime import datetime
import pytz

def find_matching_times(calendar1, calendar2):
    """
    Find matching available time slots between two calendars with Calendly-like structure.
    
    Args:
        calendar1 (dict): First calendar data in Calendly format
        calendar2 (dict): Second calendar data in Calendly format
        
    Returns:
        list: List of matching datetime objects that are available in both calendars
    """
    # Extract timezones (use the first calendar's timezone if second doesn't specify)
    tz1 = pytz.timezone(calendar1.get('availability_timezone', 'UTC'))
    tz2 = pytz.timezone(calendar2.get('availability_timezone', tz1.zone))
    
    # Create sets of available times from both calendars
    times1 = set()
    times2 = set()
    
    # Helper function to extract times from a calendar
    def extract_times(calendar, time_set):
        for day in calendar.get('days', []):
            if day['status'] == 'available' and day.get('enabled', True):
                for spot in day.get('spots', []):
                    if spot['status'] == 'available' and spot.get('invitees_remaining', 0) > 0:
                        # Parse the ISO format time string
                        time = datetime.fromisoformat(spot['start_time'])
                        time_set.add(time)
    
    # Extract times from both calendars
    extract_times(calendar1, times1)
    extract_times(calendar2, times2)
    
    # Find intersection of available times
    matching_times = sorted(times1.intersection(times2))
    
    return matching_times

def format_matches(matching_times):
    """
    Format matching times in a way suitable for passing to an LLM scheduling assistant.
    
    Args:
        matching_times (list): List of datetime objects
        
    Returns:
        str: Formatted string of available times
    """
    # Format each time with ISO 8601 format and a human-readable description
    formatted_times = []
    for time in matching_times:
        iso_format = time.strftime("%Y-%m-%dT%H:%M:%S-07:00")
        readable_format = time.strftime("%A, %B %d, %Y at %I:%M %p")
        formatted_times.append(f"{readable_format} ({iso_format})")
    
    # Join the times with line breaks for better readability in the prompt
    return "\n".join(formatted_times)

# Example usage
try:
    matches = find_matching_times(mock_calendar, calendly_data)
    formatted_matches = format_matches(matches)
    
    print(f"\nFound {len(formatted_matches)} matching time slots:")
 
        
except Exception as e:
    print(f"Error occurred: {str(e)}")

formatted_matches


Found 3870 matching time slots:


'Monday, March 17, 2025 at 04:30 PM (2025-03-17T16:30:00-07:00)\nTuesday, March 18, 2025 at 11:30 AM (2025-03-18T11:30:00-07:00)\nTuesday, March 18, 2025 at 12:00 PM (2025-03-18T12:00:00-07:00)\nTuesday, March 18, 2025 at 12:30 PM (2025-03-18T12:30:00-07:00)\nTuesday, March 18, 2025 at 01:00 PM (2025-03-18T13:00:00-07:00)\nTuesday, March 18, 2025 at 01:30 PM (2025-03-18T13:30:00-07:00)\nTuesday, March 18, 2025 at 02:00 PM (2025-03-18T14:00:00-07:00)\nTuesday, March 18, 2025 at 02:30 PM (2025-03-18T14:30:00-07:00)\nTuesday, March 18, 2025 at 03:00 PM (2025-03-18T15:00:00-07:00)\nTuesday, March 18, 2025 at 03:30 PM (2025-03-18T15:30:00-07:00)\nTuesday, March 18, 2025 at 04:00 PM (2025-03-18T16:00:00-07:00)\nTuesday, March 18, 2025 at 04:30 PM (2025-03-18T16:30:00-07:00)\nWednesday, March 19, 2025 at 09:00 AM (2025-03-19T09:00:00-07:00)\nWednesday, March 19, 2025 at 09:30 AM (2025-03-19T09:30:00-07:00)\nWednesday, March 19, 2025 at 10:00 AM (2025-03-19T10:00:00-07:00)\nWednesday, March 19

In [5]:
response_schema = ResponseSchema(
            name="suggested_time",
            description="The suggested meeting time in ISO 8601 format with UTC -07:00 timezone",
            type="string"
        )
        
parser = StructuredOutputParser.from_response_schemas([response_schema])
format_instructions = parser.get_format_instructions()
        
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"suggested_time": string  // The suggested meeting time in ISO 8601 format with UTC -07:00 timezone\n}\n```'

In [6]:
prompt_template = scheduling_prompt()
messages = prompt_template.format_messages(
            overlapping_availability=formatted_matches,
            format_instructions=format_instructions
        )
        
        
llm = ChatOpenAI(
            model_name="gpt-4o-mini",
            temperature=0,
            openai_api_key=os.getenv("OPENAI_API_KEY")
        )
        
response = llm.invoke(messages)
parsed_response = parser.parse(response.content)
suggested_time = parsed_response['suggested_time']
print(messages)

/tmp/ipykernel_160300/4183445104.py:8: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


2025-03-15 20:01:09,570 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
[HumanMessage(content='\n    You are a scheduling assistant. Please analyze the following available meeting times and suggest the best option.\n\n    Available Times:\n    Monday, March 17, 2025 at 04:30 PM (2025-03-17T16:30:00-07:00)\nTuesday, March 18, 2025 at 11:30 AM (2025-03-18T11:30:00-07:00)\nTuesday, March 18, 2025 at 12:00 PM (2025-03-18T12:00:00-07:00)\nTuesday, March 18, 2025 at 12:30 PM (2025-03-18T12:30:00-07:00)\nTuesday, March 18, 2025 at 01:00 PM (2025-03-18T13:00:00-07:00)\nTuesday, March 18, 2025 at 01:30 PM (2025-03-18T13:30:00-07:00)\nTuesday, March 18, 2025 at 02:00 PM (2025-03-18T14:00:00-07:00)\nTuesday, March 18, 2025 at 02:30 PM (2025-03-18T14:30:00-07:00)\nTuesday, March 18, 2025 at 03:00 PM (2025-03-18T15:00:00-07:00)\nTuesday, March 18, 2025 at 03:30 PM (2025-03-18T15:30:00-07:00)\nTuesday, March 18, 2025 at 04:00 PM (2025-03-18T16:00:00-0

In [7]:
suggested_time

'2025-03-18T11:30:00-07:00?month=2025-03&date=2025-03-18'

In [8]:


calendly_url = "https://calendly.com/robertjandali/30min"
name = "John Doe"
email = "john@doe.com"
phone = "5109198404"
additional_info = "This is a test booking"

# Get mock calendar data


# Get suggested time from LLM
suggested_time = get_suggested_time(formatted_matches)
print(suggested_time)
# Create final booking URL

final_url = create_booking_url(calendly_url, suggested_time)
final_url
# Set up Selenium with reCAPTCHA solver and check for reCAPTCHA


2025-03-15 20:01:41,511 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-17T16:30:00-07:00?month=2025-03&date=2025-03-17


'https://calendly.com/robertjandali/30min/2025-03-17T16:30:00-07:00?month=2025-03&date=2025-03-17'

In [51]:
print(suggested_time)

2025-03-18T09:00:00-07:00?month=2025-03&date=2025-03-18
